# Compare 20 S.M. models totals

Use plotly to display results from a list of supernovae model simulations processed with sspike.

In [1]:
from os import listdir
from os.path import isfile

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

from sspike.env import sspike_dir

## Get file paths to visible totals

Simulations can be run via: `sspike progenitors/m20d10kpc.json`.

Notes:
- There seems to be something wrong with the Fornax model numbers.
- This notebook may not work right if there are similar simulations in the bin_dir.

In [2]:
# List simulations of 20 solar masses at 10 kpc. (m20d10kpc.json)
# Something seems off with the Fornax models.
with_fornax = False

if with_fornax:
    sn_names = ['F21-20.0', 
                'K20-00-00', 'K20-10-12', 'K20-10-13',
                'N13-20-20-100', 'N13-20-20-200', 'N13-20-20-300', 
                'T14-20.0',
                'W20-20.0-1.23', 'W20-20.0-1.25', 'W20-20.0-1.27']
else:
    sn_names = ['K20-00-00', 'K20-10-12', 'K20-10-13',
                'N13-20-20-100', 'N13-20-20-200', 'N13-20-20-300', 
                'T14-20.0',
                'W20-20.0-1.23', 'W20-20.0-1.25', 'W20-20.0-1.27']

xforms = ['NT', 'aMSW-I', 'aMSW-N']

# List for filepaths to totals files tabulated using sspike.beer.tab()
files = []

# Folder containing simulation results for each model.
for name in sn_names:
    for x in xforms:
        sn_dir = f"{sspike_dir}/kamland/{name}/10.0kpc-{x}"
        for bin_dir in listdir(sn_dir):
            if bin_dir == "b1s0.0e0.5":
                continue
            file_name = f"{sn_dir}/{bin_dir}/totals_vis_0.csv"
            if isfile(file_name):
                files.append(file_name)

## Compile data

Read each file and add event rates to a single DataFrame.

In [4]:
# Columns for simulation type and events per channel.
column_names = ['sn_name', 'model', 'channel', 'counts', 'ratio', 'xform']

events = pd.DataFrame(columns=column_names)
# Append 1 row for each channel.  I don't care if it's not efficient.  Who's reading this?
for i, file in enumerate(files):
    dirs = file.split('/')
    j = dirs.index('kamland')
    name = dirs[j+1]
    xform = dirs[j+2].split('-')[-1]
    df = pd.read_csv(file, sep=" ")
    model = df[:1]['model'].values[0]
    ibd = df[df['channel']=='ibd']['events'].values[0]
    for chan in df['channel']:
        N = df[df['channel']==chan]['events'].values[0]
        ratio = N / ibd
        row = {'sn_name': name, 'model': model, 'channel': chan, 'counts': N, 'ratio': ratio, 'xform': xform}
        events = events.append(row , ignore_index=True)

## Plotly

In [5]:
bars = px.bar(events, x='channel', y='counts', 
              color='sn_name', barmode='group', log_y=True)
bars.layout.bargroupgap = 0.2
bars.layout.font = dict(size=14, family="Times New Roman")
bars.layout.bargap = 0.1
bars.layout.bargroupgap = 0.05
bars.layout.width = 740
bars.layout.height = 400
bars.layout.title = f'Comaprison of 20 S.M. models @ 10 kpc in KamLAND'
bars

In [5]:
bars.write_image('../out/plots/m20.png', scale=3)
bars.write_html('../out/plots/m20.html')

In [6]:
bars = px.bar(events, x='channel', y='ratio', 
              color='sn_name', barmode='group', log_y=True)
bars.layout.bargroupgap = 0.2
bars.layout.font = dict(size=14, family="Times New Roman")
bars.layout.bargap = 0.1
bars.layout.bargroupgap = 0.05
bars.layout.width = 740
bars.layout.height = 400
bars.layout.title = f'Comaprison of 20 S.M. models @ 10 kpc in KamLAND'
bars

In [13]:
bars.write_image('../out/plots/m20_ratio.png', scale=3)
bars.write_html('../out/plots/m20_ratio.html')

In [7]:
bars = px.bar(events, x='xform', y='ratio', 
              color='sn_name', barmode='group',
              facet_col='channel', 
              category_orders={"mass": [13, 20, 30, 50],
                               "metal": [0.02, 0.004]},
              log_y=True)
bars.layout.bargroupgap = 0.2
bars.layout.font = dict(size=14, family="Times New Roman")
bars.layout.bargap = 0.1
bars.layout.bargroupgap = 0.05
bars.layout.width = 1000
bars.layout.height = 400
bars.layout.title = f'Nakazato_2013 @ 10 kpc in KamLAND'
bars

In [38]:
bars.write_image('../out/plots/m20_msw.png', scale=3)
bars.write_html('../out/plots/m20_msw.html')

In [8]:
scat = px.scatter(events[events['channel']!='ibd'][events['xform']=='NT'], x='sn_name', y='ratio', 
              color='channel',
              symbol='xform',
              )
scat.layout.bargroupgap = 0.2
scat.layout.font = dict(size=18, family="Times New Roman")
scat.layout.width = 740
scat.layout.height = 700
scat.layout.title = f'Comaprison of 20 S.M. models @ 10 kpc in KamLAND'
scat.update_traces(marker_size=16)
scat

/var/folders/bs/t_j9r03n3h39lmjyz9hr6b540000gn/T/ipykernel_13068/2250513182.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [53]:
scat.write_image('../out/plots/m20_msw_scat.png', scale=3)
scat.write_html('../out/plots/m20_msw_scat.html')

In [9]:
scat = px.scatter(events, x='sn_name', y='counts', 
              color='channel',
              symbol='xform',
              log_y=True
              )
scat.layout.bargroupgap = 0.2
scat.layout.font = dict(size=18, family="Times New Roman")
scat.layout.width = 740
scat.layout.height = 700
scat.layout.title = f'Comaprison of 20 S.M. models @ 10 kpc in KamLAND'
scat.update_traces(marker_size=16)

In [10]:
scat = px.scatter(events[events['channel']!='ibd'], x='sn_name', y='counts', 
              color='channel',
              symbol='xform',
              log_y=False
              )
scat.layout.bargroupgap = 0.2
scat.layout.font = dict(size=18, family="Times New Roman")
scat.layout.width = 740
scat.layout.height = 700
scat.layout.title = f'Comaprison of 20 S.M. models @ 10 kpc in KamLAND'
scat.update_traces(marker_size=16)